# Prerequisites
- Python 3.10.7 or greater
- A Google Cloud project
- A Google account with Gmail enabled.

# Steps

1. Create Google Cloud Project
2. Enable Email API
3. Configure the OAuth consent screen: Menu  -> Google Auth platform -> Branding    
4. Authorize credentials for a desktop application
5. Create Client: Menu > Google Auth platform > Clients
6. Save the downloaded JSON file as client_secret.json, and move the file to your working directory.
7. Install the Google client library: 
 ```
 pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

 ```


create service and authenticate

In [4]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

def create_service(client_secret_file, api_name, api_version, *scopes, prefix=''):
    
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes][0]

    creds = None
    working_dir = os.getcwd()
    token_dir = 'token files'
    token_file = f'token_{API_SERVICE_NAME}_{API_VERSION}{prefix}.json'

    ### Check if token dir exists first, if not, create the folder
    if not os.path.exists(os.path.join(working_dir, token_dir)):
        os.mkdir(os.path.join(working_dir, token_dir))

    if os.path.exists(os.path.join(working_dir, token_dir, token_file)):
        creds = Credentials.from_authorized_user_file(
            os.path.join(working_dir, token_dir, token_file), SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(os.path.join(working_dir, token_dir, token_file), 'w') as token:
            token.write(creds.to_json())

    try:
        service = build(
            API_SERVICE_NAME,
            API_VERSION,
            credentials=creds,
            static_discovery=False
        )
        print(f'{API_SERVICE_NAME} {API_VERSION} service created successfully')
        return service

    except Exception as e:
        print(e)
        print(f'Failed to create service instance for {API_SERVICE_NAME}')
        os.remove(os.path.join(working_dir, token_dir, token_file))
        return None


call the service

In [2]:
client_secret_file = 'client_secrets.json'
API_SERVICE_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = create_service(client_secret_file, API_SERVICE_NAME, API_VERSION, SCOPES)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=145859269760-g7hfcq4vspc41mratln2qvaq2dg9fnvr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49753%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=4h1PZNUIcIExhIOZmRf5Rb6lUJMxQ0&access_type=offline
gmail v1 service created successfully


In [3]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_credentials_validated',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 '_universe_domain',
 '_validate_credentials',
 'close',
 'new_batch_http_request',
 'users']

1. fetch emails

In [9]:
import os
import base64
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from googleapiclient import errors


def init_gmail_service(client_file, api_name='gmail', api_version='v1',
                       scopes=['https://mail.google.com/']):
    
    return create_service(client_file, api_name, api_version, scopes) # call created service function

# extract the email body from the payload
def _extract_body(payload):
    body = '<Text body not available>'

    if 'parts' in payload:
        for part in payload['parts']:
            if part.get('mimeType') == 'multipart/alternative':
                for subpart in part.get('parts', []):
                    if (
                        subpart.get('mimeType') == 'text/plain'
                        and 'data' in subpart.get('body', {})
                    ):
                        body = base64.urlsafe_b64decode(
                            subpart['body']['data']
                        ).decode('utf-8')
                        break

            elif (
                part.get('mimeType') == 'text/plain'
                and 'data' in part.get('body', {})
            ):
                body = base64.urlsafe_b64decode(
                    part['body']['data']
                ).decode('utf-8')
                break

    elif 'body' in payload and 'data' in payload['body']:
        body = base64.urlsafe_b64decode(
            payload['body']['data']
        ).decode('utf-8')

    return body

# function to get email messages with pagination and folder filtering
def get_email_messages(
    service,
    user_id='me',
    label_ids=None,
    folder_name='INBOX',
    max_results=5
):
    messages = []
    next_page_token = None

    # Resolve folder (label) name → label_id
    if folder_name:
        label_results = service.users().labels().list(userId=user_id).execute()
        labels = label_results.get('labels', [])
        folder_label_id = next(
            (label['id'] for label in labels if label['name'].lower() == folder_name.lower()),
            None
        )

        if folder_label_id:
            if label_ids:
                label_ids.append(folder_label_id)
            else:
                label_ids = [folder_label_id]
        else:
            raise ValueError(f"Folder '{folder_name}' not found.")

    # Pagination loop
    while True:
        result = (
            service.users()
            .messages()
            .list(
                userId=user_id,
                labelIds=label_ids,
                maxResults=min(500, max_results - len(messages)) if max_results else 500,
                pageToken=next_page_token
            )
            .execute()
        )

        messages.extend(result.get('messages', []))
        next_page_token = result.get('nextPageToken')

        if not next_page_token or (max_results and len(messages) >= max_results):
            break

    return messages[:max_results] if max_results else messages


def get_email_message_details(service, msg_id):
    message = service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    payload = message.get('payload')
    headers = payload.get('headers', [])

    subject = next(
        (header['value'] for header in headers if header['name'].lower() == 'subject'),
        None
    )
    if not subject:
        subject = message.get('snippet', 'No subject')

    sender = next(
        (header['value'] for header in headers if header['name'] == 'From'),
        'No sender'
    )

    recipients = next(
        (header['value'] for header in headers if header['name'] == 'To'),
        'No recipients'
    )

    snippet = message.get('snippet', 'No snippet')

    has_attachments = any(
        (part.get('filename') for part in payload.get('parts', []) if part.get('filename'))
    )

    date = next(
        (header['value'] for header in headers if header['name'] == 'Date'),
        'No date'
    )

    star = message.get('labelIds', []).count('STARRED') > 0

    label = ', '.join(message.get('labelIds', []))

    body = _extract_body(payload)

    return {
        'subject': subject,
        'sender': sender,
        'recipients': recipients,
        'body': body,
        'snippet': snippet,
        'has_attachments': has_attachments,
        'date': date,
        'star': star,
        'label': label,
    }


how to call the above service.

In [10]:
client_file = 'client_secrets.json'
service = init_gmail_service(client_file)

messages = get_email_messages(service, max_results=5, folder_name='INBOX')
messages

gmail v1 service created successfully


[{'id': '19b0a1f25115f992', 'threadId': '19b0a1f25115f992'},
 {'id': '19b09e64707ec6d9', 'threadId': '19b09e64707ec6d9'},
 {'id': '19b09d52e5019ee7', 'threadId': '19b09d52e5019ee7'},
 {'id': '19b09aceebf883fd', 'threadId': '19b09aceebf883fd'},
 {'id': '19b0971ba68a0fe8', 'threadId': '19b0971ba68a0fe8'}]

In [11]:
# print details of the first email message
if messages:
    first_msg_id = messages[0]['id']

    details = get_email_message_details(service, first_msg_id)
    if details : 
        print ('Subject:', details['subject'])
        print ('Sender:', details['sender'])
        print ('Recipients:', details['recipients'])
        print ('Body:', details['body'])
        print ('Snippet:', details['snippet'])
        print ('Has Attachments:', details['has_attachments'])
        print ('Date:', details['date'])
        print ('Starred:', details['star'])
        print ('Label:', details['label'])


    

Subject: Email Agent
Sender: Subhankar Dhar <subhankar.dhar@sjsu.edu>
Recipients: Advait Shinde <advait.shinde@sjsu.edu>, Shamathmika Shamathmika <shamathmika.shamathmika@sjsu.edu>
Body: Advait,

I had a meeting with Shamathmika and she will work with you to create the
email agent. The goal is to build a small email agent that:

   - Reads incoming emails
   - Uses your *existing chatbot* to draft replies
   - Lets a *human review/edit/approve*
   - Sends the email automatically after approval

Integrate an email agent that can process and draft responses to common
student inquiries, potentially using LLMs to automate repetitive tasks.
Get in touch with Shamathmika for the POC.

-- 
Subhankar Dhar, Ph.D.
Professor
School of Information Systems & Technology
San Jose State University
San Jose, CA 95192
Phone: 408 924 3499
Email; subhankar.dhar@sjsu.edu
Web: http://www.sjsu.edu/people/subhankar.dhar/

Snippet: Advait, I had a meeting with Shamathmika and she will work with you to create t

2. send an email with attachments